## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 2nd Feb. 9th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-3-2025,Condo/Co-op,6838 Willow Wood Dr #606,Boca Raton,FL,33434.0,450000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/6838-Will...,Beaches MLS,RX-11048353,N,Y,26.376593,-80.150481


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-03    15
2025-02-07    15
2025-02-05    14
2025-02-04    12
2025-02-06    11
2025-02-10     3
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
2,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923
21,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Beaches MLS,RX-11025344,N,Y,26.700385,-80.033799
49,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-11018594,N,Y,26.807721,-80.034589
61,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-10958157,N,Y,26.807721,-80.034589
33,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Beaches MLS,RX-11032753,N,Y,26.460202,-80.063278
17,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11027594,N,Y,26.405325,-80.067556
27,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11040477,N,Y,26.348292,-80.077094
18,PAST SALE,2025-02-03,Condo/Co-op,20090 Boca West Dr #348,Boca Raton,FL,33434.0,860000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20090-Boc...,Beaches MLS,RX-11034277,N,Y,26.382399,-80.150083
4,PAST SALE,2025-02-03,Condo/Co-op,19550 Sawgrass Dr #2103,Boca Raton,FL,33434.0,757777.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/19550-Saw...,Beaches MLS,RX-11041941,N,Y,26.389709,-80.155453
20,PAST SALE,2025-02-03,Condo/Co-op,20100 Boca West Dr #128,Boca Raton,FL,33434.0,750000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20100-Boc...,MARMLS,A11710040,N,Y,26.382399,-80.150083


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,...,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty
1,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,...,-80.033799,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Lisa Cregan,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None
2,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,...,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Matthew Walsh,Douglas Elliman (Jupiter),Timothy Bouchard,Douglas Elliman (Jupiter)
3,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,...,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate
4,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,...,-80.063278,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None
5,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,...,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate
6,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,...,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Michael Klein,"United Realty Group, Inc",None,None,Jessica Schuble,Serhant,None,None
7,PAST SALE,2025-02-03,Condo/Co-op,20090 Boca West Dr #348,Boca Raton,FL,33434.0,860000.0,3.0,2.5,...,-80.150083,https://www.redfin.com/FL/Boca-Raton/20090-Boc...,Lisa Carlin,Boca West Realty LLC,Judith Romanow,Boca West Realty LLC,Ellen Klein,Compass Florida LLC,None,None
8,PAST SALE,2025-02-03,Condo/Co-op,19550 Sawgrass Dr #2103,Boca Raton,FL,33434.0,757777.0,2.0,2.0,...,-80.155453,https://www.redfin.com/FL/Boca-Raton/19550-Saw...,Roberta Feuerstein,Lang Realty/ BR,Marci Imber,Lang Realty/ BR,Ira Steven Merritt,The Keyes Company,None,None
9,PAST SALE,2025-02-03,Condo/Co-op,20100 Boca West Dr #128,Boca Raton,FL,33434.0,750000.0,3.0,2.5,...,-80.150083,https://www.redfin.com/FL/Boca-Raton/20100-Boc...,Patricia Pearcy,Elite International Realty Inc,Geisy Valente,Elite International Realty Inc,MAR NON-MEMBER,NO MLS PARTICIPATION (MAR),None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

70


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$504,417


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$319


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$35,309,177


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
89
Input Previous Week Condo Average Sales Price:
688181
Input Previous Week Condo Average PSF:
377
Input Previous Week Condo Sales total (ex: 198_000_000)
61_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange
1,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,Winthrop House Condo,1588.0,NaN,1970.0,NaN,1801.0,2394.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Beaches MLS,RX-11025344,N,Y,26.700385,-80.033799,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Lisa Cregan,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,2,blue
2,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,Eastpointe Condo,2715.0,NaN,1981.0,NaN,571.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-11018594,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Matthew Walsh,Douglas Elliman (Jupiter),Timothy Bouchard,Douglas Elliman (Jupiter),3,blue
3,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,Eastpointe 1,2250.0,NaN,1978.0,NaN,589.0,2478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-10958157,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,4,blue
4,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Bar Harbour Condo,1515.0,NaN,1967.0,NaN,858.0,1823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Beaches MLS,RX-11032753,N,Y,26.460202,-80.063278,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,5,blue
5,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,Regency Highland Club Condo,1893.0,NaN,1976.0,NaN,660.0,1291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11027594,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue
6,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,Mizner Court Condo,1706.0,NaN,1988.0,NaN,654.0,2957.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11040477,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Michael Klein,"United Realty Group, Inc",None,None,Jessica Schuble,Serhant,None,None,7,blue
7,PAST SALE,2025-02-03,Condo/Co-op,20090 Boca West Dr #348,Boca Raton,FL,33434.0,860000.0,3.0,2.5,Laurel Oaks,2200.0,NaN,1981.0,NaN,391.0,1454.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20090-Boc...,Beaches MLS,RX-11034277,N,Y,26.382399,-80.150083,https://www.redfin.com/FL/Boca-Raton/20090-Boc...,Lisa Carlin,Boca West Realty LLC,Judith Romanow,Boca West Realty LLC,E

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_02102024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'Presidential Place Condo at 800 S Ocean Blvd #103 in Boca Raton'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange,Presidential Place Condo at 800 S Ocean Blvd #...
1,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,Winthrop House Condo,1588.0,NaN,1970.0,NaN,1801.0,2394.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Beaches MLS,RX-11025344,N,Y,26.700385,-80.033799,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Lisa Cregan,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,2,blue,Winthrop House Condo at 100 Worth Ave #602 in ...
2,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,Eastpointe Condo,2715.0,NaN,1981.0,NaN,571.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-11018594,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Matthew Walsh,Douglas Elliman (Jupiter),Timothy Bouchard,Douglas Elliman (Jupiter),3,blue,Eastpointe Condo at 5380 N Ocean Dr Unit 16j i...
3,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,Eastpointe 1,2250.0,NaN,1978.0,NaN,589.0,2478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-10958157,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,4,blue,Eastpointe 1 at 5380 N Ocean Dr Unit 22 - D in...
4,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Bar Harbour Condo,1515.0,NaN,1967.0,NaN,858.0,1823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Beaches MLS,RX-11032753,N,Y,26.460202,-80.063278,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,5,blue,Bar Harbour Condo at 86 Macfarlane Dr Ph C in ...
5,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,Regency Highland Club Condo,1893.0,NaN,1976.0,NaN,660.0,1291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11027594,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
6,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,Mizner Court Condo,1706.0,NaN,1988.0,NaN,654.0,2957.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11040477,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Michael Klein,"United Realty Group, Inc",None,None,Jessica Schuble,Serhant,None,None,7,blue,Mizner Court Condo at 100 SE 5th Ave #209 in B...
7,PAST SALE,2025-

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Presidential Place Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $750,000 to $7,750,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Singer Island, Highland Beach, Palm Beach, Delray Beach, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $35,309,177 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 89 condo sales totaling $61,300,000.

Last week’s units sold for an average of $504,417, lower than t

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange,Presidential Place Condo at 800 S Ocean Blvd #...
1,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,Winthrop House Condo,1588.0,NaN,1970.0,NaN,1801.0,2394.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Beaches MLS,RX-11025344,N,Y,26.700385,-80.033799,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Lisa Cregan,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,2,blue,Winthrop House Condo at 100 Worth Ave #602 in ...
2,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,Eastpointe Condo,2715.0,NaN,1981.0,NaN,571.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-11018594,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Matthew Walsh,Douglas Elliman (Jupiter),Timothy Bouchard,Douglas Elliman (Jupiter),3,blue,Eastpointe Condo at 5380 N Ocean Dr Unit 16j i...
3,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,Eastpointe 1,2250.0,NaN,1978.0,NaN,589.0,2478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-10958157,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,4,blue,Eastpointe 1 at 5380 N Ocean Dr Unit 22 - D in...
4,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Bar Harbour Condo,1515.0,NaN,1967.0,NaN,858.0,1823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Beaches MLS,RX-11032753,N,Y,26.460202,-80.063278,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,5,blue,Bar Harbour Condo at 86 Macfarlane Dr Ph C in ...
5,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,Regency Highland Club Condo,1893.0,NaN,1976.0,NaN,660.0,1291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11027594,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
6,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,Mizner Court Condo,1706.0,NaN,1988.0,NaN,654.0,2957.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11040477,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Michael Klein,"United Realty Group, Inc",None,None,Jessica Schuble,Serhant,None,None,7,blue,Mizner Court Condo at 100 SE 5th Ave #209 in B...
7,PAST SALE,2025-

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Presidential Place Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $750,000 to $7,750,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Singer Island, Highland Beach, Palm Beach, Delray Beach, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $35,309,177 million from Feb. 2nd to Feb. 9th. The previous week, brokers closed 89 condo sales totaling $61,300,000.

Last week’s units sold for an average of $504,417, lower than t

In [58]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/800-S-Ocean-Blvd-33432/unit-103/home/42322144


In [52]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/100-Worth-Ave-33480/unit-602/home/42435282


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boca-Raton/20100-Boca-West-Dr-33434/unit-128/home/42254093


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/19550-Sawgrass-Dr-33434/unit-2103/home/42247520


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/100-Worth-Ave-33480/unit-602/home/42435282


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/20100-Boca-West-Dr-33434/unit-128/home/42254093


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-10,Condo/Co-op,800 S Ocean Blvd #103,Boca Raton,FL,33432.0,7750000.0,4.0,4.5,Presidential Place Condo,4747.0,NaN,1994.0,NaN,1633.0,7022.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Beaches MLS,RX-11060822,N,Y,26.339913,-80.070923,https://www.redfin.com/FL/Boca-Raton/800-S-Oce...,Senada Adzem,Douglas Elliman,None,None,Erin Omiatek,One Sotheby's International Realty,Zina Cortese,One Sotheby's International Realty,1,orange,Presidential Place Condo at 800 S Ocean Blvd #...
1,PAST SALE,2025-02-10,Condo/Co-op,100 Worth Ave #602,Palm Beach,FL,33480.0,2859200.0,2.0,2.0,Winthrop House Condo,1588.0,NaN,1970.0,NaN,1801.0,2394.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Beaches MLS,RX-11025344,N,Y,26.700385,-80.033799,https://www.redfin.com/FL/Palm-Beach/100-Worth...,Lisa Cregan,"Sotheby's Intl. Realty, Inc.",None,None,Suzanne Trapani Frisbie,The Corcoran Group,None,None,2,blue,Winthrop House Condo at 100 Worth Ave #602 in ...
2,PAST SALE,2025-02-03,Condo/Co-op,5380 N Ocean Dr Unit 16j,Singer Island,FL,33404.0,1550000.0,3.0,3.5,Eastpointe Condo,2715.0,NaN,1981.0,NaN,571.0,2524.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-11018594,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Vincent Marotta,Illustrated Properties LLC (Co,None,None,Matthew Walsh,Douglas Elliman (Jupiter),Timothy Bouchard,Douglas Elliman (Jupiter),3,blue,Eastpointe Condo at 5380 N Ocean Dr Unit 16j i...
3,PAST SALE,2025-02-07,Condo/Co-op,5380 N Ocean Dr Unit 22 - D,Singer Island,FL,33404.0,1325000.0,3.0,3.0,Eastpointe 1,2250.0,NaN,1978.0,NaN,589.0,2478.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Beaches MLS,RX-10958157,N,Y,26.807721,-80.034589,https://www.redfin.com/FL/Riviera-Beach/5380-N...,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,Tony Sacco,Singer Island Real Estate,Dawn Sacco Paskalides,Singer Island Real Estate,4,blue,Eastpointe 1 at 5380 N Ocean Dr Unit 22 - D in...
4,PAST SALE,2025-02-06,Condo/Co-op,86 Macfarlane Dr Ph C,Delray Beach,FL,33483.0,1300000.0,2.0,2.0,Bar Harbour Condo,1515.0,NaN,1967.0,NaN,858.0,1823.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Beaches MLS,RX-11032753,N,Y,26.460202,-80.063278,https://www.redfin.com/FL/Delray-Beach/86-Mac-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,5,blue,Bar Harbour Condo at 86 Macfarlane Dr Ph C in ...
5,PAST SALE,2025-02-04,Condo/Co-op,3912 S Ocean Blvd #1406,Highland Beach,FL,33487.0,1250000.0,3.0,3.0,Regency Highland Club Condo,1893.0,NaN,1976.0,NaN,660.0,1291.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3912-...,Beaches MLS,RX-11027594,N,Y,26.405325,-80.067556,https://www.redfin.com/FL/Highland-Beach/3912-...,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Regency Highland Club Condo at 3912 S Ocean Bl...
6,PAST SALE,2025-02-03,Condo/Co-op,100 SE 5th Ave #209,Boca Raton,FL,33432.0,1115600.0,2.0,2.0,Mizner Court Condo,1706.0,NaN,1988.0,NaN,654.0,2957.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Beaches MLS,RX-11040477,N,Y,26.348292,-80.077094,https://www.redfin.com/FL/Boca-Raton/100-SE-5t...,Michael Klein,"United Realty Group, Inc",None,None,Jessica Schuble,Serhant,None,None,7,blue,Mizner Court Condo at 100 SE 5th Ave #209 in B...
7,PAST SALE,2025-

## Time on Market Calculator

In [60]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 30) ## List (Earlier) date
date2 = datetime(2025, 2, 10) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

133


## Clean CSV for Datawrapper Chart

In [61]:
chart_df = df_top_ten

In [62]:
chart_df = chart_df.fillna(" ")

In [63]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [64]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [65]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [66]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [67]:
chart_df['$/SQUARE FEET'].astype(float)

0    1633.0
1    1801.0
2     571.0
3     589.0
4     858.0
5     660.0
6     654.0
7     391.0
8     454.0
9     341.0
Name: $/SQUARE FEET, dtype: float64

In [68]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [69]:
csv_date_string = today.strftime("%m_%d_%Y")

In [70]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [71]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-10,800 S Ocean Blvd #103 in Boca Raton,"$7,750,000",4,4.5,"4,747",1994,"$1,633",Beaches MLS,RX-11060822,Senada Adzem Douglas Elliman,Erin Omiatek One Sotheby's International Realt...
1,2025-02-10,100 Worth Ave #602 in Palm Beach,"$2,859,200",2,2.0,"1,588",1970,"$1,801",Beaches MLS,RX-11025344,"Lisa Cregan Sotheby's Intl. Realty, Inc.",Suzanne Trapani Frisbie The Corcoran Group
2,2025-02-03,5380 N Ocean Dr Unit 16j in Singer Island,"$1,550,000",3,3.5,"2,715",1981,$571,Beaches MLS,RX-11018594,Vincent Marotta Illustrated Properties LLC (Co...,Matthew Walsh Douglas Elliman (Jupiter) Timoth...
3,2025-02-07,5380 N Ocean Dr Unit 22 - D in Singer Island,"$1,325,000",3,3.0,"2,250",1978,$589,Beaches MLS,RX-10958157,Tony Sacco Singer Island Real Estate Dawn Sacc...,Tony Sacco Singer Island Real Estate Dawn Sacc...
4,2025-02-06,86 Macfarlane Dr Ph C in Delray Beach,"$1,300,000",2,2.0,"1,515",1967,$858,Beaches MLS,RX-11032753,Geoffrey Braboy Compass Florida LLC,Geoffrey Braboy Compass Florida LLC
5,2025-02-04,3912 S Ocean Blvd #1406 in Highland Beach,"$1,250,000",3,3.0,"1,893",1976,$660,Beaches MLS,RX-11027594,Matthew Bachrad William Raveis Real Estate Dan...,Matthew Bachrad William Raveis Real Estate Dan...
6,2025-02-03,100 SE 5th Ave #209 in Boca Raton,"$1,115,600",2,2.0,"1,706",1988,$654,Beaches MLS,RX-11040477,"Michael Klein United Realty Group, Inc",Jessica Schuble Serhant
7,2025-02-03,20090 Boca West Dr #348 in Boca Raton,"$860,000",3,2.5,"2,200",1981,$391,Beaches MLS,RX-11034277,Lisa Carlin Boca West Realty LLC Judith Romano...,Ellen Klein Compass Florida LLC
8,2025-02-03,19550 Sawgrass Dr #2103 in Boca Raton,"$757,777",2,2.0,"1,670",1980,$454,Beaches MLS,RX-11041941,Roberta Feuerstein Lang Realty/ BR Marci Imber...,Ira Steven Merritt The Keyes Company
9,2025-02-03,20100 Boca West Dr #128 in Boca Raton,"$750,000",3,2.5,"2,200",1980,$341,MARMLS,A11710040,Patricia Pearcy Elite International Realty Inc...,MAR NON-MEMBER NO MLS PARTICIPATION (MAR)
